In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, concatenate
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:

def load_images_from_dir(directory):
    images = []
    labels = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            img = cv2.imread(os.path.join(directory, filename))
            # Resizing 
            img = cv2.resize(img, (224, 224))
            img = img.astype(np.float32) / 255.0
            images.append(img)
            labels.append(directory.split('/')[-1])  
    return np.array(images), np.array(labels)

In [3]:
#variable assignment
ad_creatives_dir = "ad-creatives"
non_ad_creatives_dir = "non_ad-creatives"

In [ ]:
ad_images, ad_labels = load_images_from_dir(ad_creatives_dir)
non_ad_images, non_ad_labels = load_images_from_dir(non_ad_creatives_dir)

In [5]:
all_images = np.concatenate((ad_images, non_ad_images), axis=0)
all_labels = np.concatenate((ad_labels, non_ad_labels), axis=0)

In [6]:
indices = np.arange(all_images.shape[0])
np.random.shuffle(indices)
all_images = all_images[indices]
all_labels = all_labels[indices]

In [7]:
print("Shape of all images:", all_images.shape)
print("Shape of all labels:", all_labels.shape)

Shape of all images: (0,)
Shape of all labels: (0,)


In [8]:
# data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,      
    width_shift_range=0.1,  
    height_shift_range=0.1, 
    shear_range=0.2,        
    zoom_range=0.2,         
    horizontal_flip=True,   
    fill_mode='nearest'     
)

In [9]:
def augment_images(images):
    augmented_images = []
    for image in images:
        image = np.expand_dims(image, axis=0)  
        augmented_image = next(datagen.flow(image))[0]
        augmented_images.append(augmented_image)
    return np.array(augmented_images)

In [10]:
augmented_ad_images = augment_images(ad_images)

In [11]:
augmented_non_ad_images = augment_images(non_ad_images)

In [12]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [13]:
#categorical to numerical
label_encoder = LabelEncoder()
all_labels_encoded = label_encoder.fit_transform(all_labels)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(all_images, all_labels_encoded, test_size=0.2, random_state=42)

ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of X_val:", X_val.shape)
print("Shape of y_val:", y_val.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)

ResNet would be a good choice here for model building.

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, concatenate

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)  
predictions = Dense(1, activation='sigmoid')(x)

In [ ]:

model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:

import tensorflow as tf

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay

initial_learning_rate = 0.001
decay_steps = 1000
decay_rate = 0.9

learning_rate_schedule = ExponentialDecay(
    initial_learning_rate, decay_steps, decay_rate, staircase=True
)
optimizer = Adam(learning_rate=learning_rate_schedule)

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Now training the model

In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=10,  
    batch_size=32,  
    validation_data=(X_val, y_val),
    verbose=1
)

In [ ]:
print("hello")

In [ ]:

loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


Manually checking using few examples

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
sample_indices = np.random.choice(X_test.shape[0], size=5, replace=False)
sample_images = X_test[sample_indices]
sample_labels = y_test[sample_indices]

In [ ]:

predictions = model.predict(sample_images)
predicted_labels = np.round(predictions).astype(int)

In [ ]:

for i in range(len(sample_indices)):
    plt.imshow(sample_images[i])
    plt.title(f"Predicted: {predicted_labels[i]}, True: {sample_labels[i]}")
    plt.axis('off')
    plt.show()

In [ ]:

def map_labels_to_strings(labels):
    label_map = {0: "ad", 1: "non-ad"}
    return [label_map[label] for label in labels]

In [ ]:
def display_sample_images(images, labels, predictions):
    for i in range(len(images)):
        plt.imshow(images[i])
        plt.title(f"Predicted: {predictions[i]}, True: {labels[i]}")
        plt.axis('off')
        plt.show()
num_samples = 5

In [ ]:
while True:
    sample_indices = np.random.choice(X_test.shape[0], size=num_samples, replace=False)
    sample_images = X_test[sample_indices]
    sample_labels = y_test[sample_indices]

    predictions = model.predict(sample_images)
    predicted_labels = np.round(predictions).astype(int)

    display_sample_images(sample_images, sample_labels, predicted_labels)

    user_input = input("Do you want to continue checking? (yes/no): ").lower()
    if user_input != 'yes':
        break

Add probability scores 

In [ ]:
print("till now good")

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Define a function to display sample images with predictions and probability scores
def display_sample_images_with_prob(images, labels, predictions, prob_scores):
    for i in range(len(images)):
        plt.imshow(images[i])
        plt.title(f"Predicted: {predictions[i]}, Probability: {prob_scores[i][0]:.2f}, True: {labels[i]}")
        plt.axis('off')
        plt.show()

y_pred_prob = model.predict(X_test)
y_pred = np.round(y_pred_prob).astype(int)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

# Display evaluation metrics
print("Evaluation Metrics:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("Confusion Matrix:")
print(conf_matrix)

# Display sample images with predictions and probability scores
display_sample_images_with_prob(X_test, y_test, y_pred, y_pred_prob)
